In [ ]:
%matplotlib qt
import numpy as np
import lmfit
from bluesky import RunEngine
from ophyd.sim import SynGauss, SynAxis, motor1
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky.callbacks import LiveFitPlot, LiveFit, LivePlot
from bluesky.plans import scan
from matplotlib.pyplot import ion, subplots
from bluesky.utils import install_nb_kicker
from scipy.special import erf
install_nb_kicker()
ion()

## RunEngine

In [ ]:
RE = RunEngine({})
bec = BestEffortCallback()
RE.subscribe(bec)

## Device simulado

##### É possível criar devices que simulam o comportamento de algum detector ou fenômeno específico, nesse caso há um modelo de um detector que realiza a integral de um sinal com perfil gaussiano. Na prática, qualquer tipo de device pode ser implementado para simular experimentos ou scans na linha de luz.

In [ ]:
import numpy as np #Hide this import here just for now

def gaussian_integral(x, peak, sigma, center):
    return peak * sigma * np.sqrt(np.pi / 2) * (erf((x - center) / (np.sqrt(2) * sigma)) - erf((-5 - center) / (np.sqrt(2) * sigma)))

class SynKnifeDetector(SynGauss):
    
    def __init__(self, name, motor, motor_field, center, Imax, *, random_state=None, **kwargs):
        super().__init__(name, motor, motor_field, center, Imax, **kwargs)

    def _compute(self):
        m = self._motor.read()[self._motor_field]["value"]
        Imax = self.Imax.get()
        center = self.center.get()
        sigma = self.sigma.get()
        noise = self.noise.get()
        noise_multiplier = self.noise_multiplier.get()
        
        return gaussian_integral(m, Imax, sigma, center)
        


## Instância dos devices simulados

#### É possível definir os parâmetros na inicialização do device **detector**

In [ ]:
motor1.delay = 0.15
detector = SynKnifeDetector('detector', motor1, 'motor1', center=2, Imax=9, sigma=1)

### Plot fitting


#### Nesse caso, podemos testar funções como o LiveFitPlot e obter o resultado do fit ao final da run

In [ ]:
fig, ax = subplots()
bec.disable_plots()

    
model = lmfit.Model(gaussian_integral)

init_guess = {'peak': 5, 'sigma': 1.5, 'center': 3}


live_fit = LiveFit(model, 'detector', {'x': 'motor1'}, init_guess)
live_fit_plot = LiveFitPlot(live_fit, color='r', ax=ax, label='Fit')
live_plot = LivePlot('detector', 'motor1', marker='x', linestyle='none', ax=ax, label='Scan')

RE(
    scan([detector], motor1, -10, 10, num=40),
    [live_fit_plot, live_plot]
)

In [ ]:
live_fit.result

In [ ]:
data = db[-1].table() #Should we use databroker now?